### Bare Soil (Fractional Cover) 10m resolution
Created by Amos Bennett.<br>
Last Updated 23 Sep 20. <br>

Generates a pre and post fire base soil fractional cover geotiff file by resampling LS8 FC product to 10m resolution.

In [ ]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/home/554/ab4513/dea-notebooks/Scripts")
import dea_datahandling
from dea_datahandling import load_ard
from dea_plotting import display_map

In [ ]:
print(datacube.__version__)

In [ ]:
dc = datacube.Datacube(app="baresoil")

In [ ]:
# Set the central latitude and longitude
central_lat = -35.783333
central_lon = 148.016667
crs = 'EPSG:32755'

# Canberra 148.6547665°E 35.5655761°S
# Kosciuszko 148.3517111°E 36.1864717°S
# Tumbarumba 148.016667°E 35.783333°S

# 0.1° approximately equal to 11.1km distance.

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc) in degrees (lat, lon)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

# display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

In [ ]:
# Key dates to load pre and postfire Landsat 8 FC scenes
prefire_start = '2019-12-21'
prefire_end = '2019-12-23'
postfire_start = '2020-01-07'
postfire_end = '2020-01-08'

In [ ]:
query = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (prefire_start, prefire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-10, 10)}

prefire_ls8_fc = dc.load(product="ls8_fc_albers",
                         measurements=['BS'],
                         **query)

prefire_ls8_fc

In [ ]:
prefire_ls8 = prefire_ls8_fc.max(dim='time')
del prefire_ls8_fc

In [ ]:
prefire_bs = prefire_ls8.BS.to_dataset(name='prefire_bs')
write_geotiff(f'./prefire_BS_geotiffs/Tumbarumba_BS_prefire.tif', prefire_bs)

In [ ]:
query = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (postfire_start, postfire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-10, 10)}

postfire_ls8_fc = dc.load(product="ls8_fc_albers",
                          measurements=['BS'],
                          **query)

postfire_ls8_fc

In [ ]:
postfire_ls8 = postfire_ls8_fc.max(dim='time')
del postfire_ls8_fc

In [ ]:
postfire_bs = postfire_ls8.BS.to_dataset(name='postfire_bs')
write_geotiff(f'./postfire_BS_geotiffs/Tumbarumba_BS_postfire.tif', postfire_bs)

In [ ]:
dFCB = postfire_ls8.BS - prefire_ls8.BS
dFCB_dataset = dFCB.to_dataset(name='dFCB')
write_geotiff(f'./dFBC_geotiffs/Tumbarumba_dFCB.tif', dFCB_dataset)